# GETS Indicator Saturation

This notebook demonstrates the **GETS indicator saturation** module in `regimes`, which implements
the Autometrics algorithm (Doornik, 2009; Castle, Doornik & Hendry, 2012) for detecting structural
breaks via general-to-specific model selection.

**Topics covered:**
- **SIS** (Step Indicator Saturation) — detecting level shifts in the intercept
- **IIS** (Impulse Indicator Saturation) — detecting outliers
- **MIS** (Multiplicative Indicator Saturation) — detecting coefficient shifts
- **TIS** (Trend Indicator Saturation) — detecting broken linear trends
- **Dual representation** — shifts vs regime levels
- **Model integration** — `.isat()` convenience methods on OLS, AR, ADL
- **Method comparison** — GETS vs Bai-Perron vs Markov switching
- **End-to-end workflow** — from CUSUM screening to final model

In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path("..") / "src"))

import warnings

import matplotlib.pyplot as plt
import numpy as np

import regimes as rg

warnings.filterwarnings("ignore", category=RuntimeWarning)
print(f"regimes version: {rg.__version__}")

## 1. Data Generation

We create four synthetic datasets, each designed to test a specific indicator type.

In [ ]:
rng = np.random.default_rng(42)
n = 300

# SIS data: mean shifts at t=100 and t=200
y_sis = np.concatenate([
    rng.normal(0.0, 1.0, 100),
    rng.normal(3.0, 1.0, 100),
    rng.normal(1.0, 1.0, 100),
])

# IIS data: noise with 3 large outliers
y_iis = rng.normal(0.0, 1.0, n)
y_iis[50] += 8.0
y_iis[150] += -7.0
y_iis[250] += 9.0

# MIS data: regression with slope change at t=150
x_mis = rng.standard_normal(n)
y_mis = np.zeros(n)
y_mis[:150] = 1.0 + 0.5 * x_mis[:150] + rng.normal(0, 0.5, 150)
y_mis[150:] = 1.0 + 2.0 * x_mis[150:] + rng.normal(0, 0.5, 150)

# TIS data: trend slope change at t=150
trend1 = np.arange(150) * 0.02
trend2 = trend1[-1] + np.arange(150) * (-0.01)
y_tis = np.concatenate([trend1, trend2]) + rng.normal(0, 0.3, n)

# Visualize all four datasets
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for ax, y, title, breaks in zip(
    axes.flat,
    [y_sis, y_iis, y_mis, y_tis],
    ["Mean Shifts (SIS)", "Outliers (IIS)", "Slope Change (MIS)", "Trend Break (TIS)"],
    [[100, 200], [50, 150, 250], [150], [150]],
):
    ax.plot(y, linewidth=0.8)
    for bp in breaks:
        ax.axvline(bp, color="red", linestyle="--", alpha=0.5)
    ax.set_title(title)
    ax.set_xlabel("Observation")
plt.tight_layout()
plt.show()

## 2. Step Indicator Saturation (SIS)

SIS is the workhorse of indicator saturation. It creates a step indicator for every candidate
break date (one per observation within the trimmed sample), includes them in the model, and
uses the GETS algorithm to select which ones are significant.

The retained step indicators identify **level shifts in the intercept**.

In [ ]:
result_sis = rg.isat(y_sis, sis=True, alpha=0.01)
print(result_sis.summary())

In [ ]:
# Explore the results
print(f"Break dates:            {result_sis.break_dates}")
print(f"Number of regimes:      {result_sis.n_regimes}")
print(f"Retained indicators:    {result_sis.retained_indicators}")
print(f"Candidate indicators:   {result_sis.n_indicators_initial}")
print(f"Retained:               {result_sis.n_indicators_retained}")

### Dual Representation: Shifts vs Regime Levels

Every SIS/MIS result has two equivalent representations:

1. **Shifts form** — what the regression directly estimates: an initial level plus step
   changes at each break date.
2. **Regime levels form** — cumulated from shifts: the level of each parameter in each
   regime, with standard errors propagated via the delta method.

In [ ]:
# Shifts representation: initial level + step changes
shifts = result_sis.shifts
print("Shifts Representation:")
print(f"  Initial levels: {shifts.initial_levels}")
print(f"  Shifts: {shifts.shifts}")
print()

# Regime levels representation: level in each regime
levels = result_sis.regime_levels
print("Regime Levels:")
for param, regimes in levels.param_regimes.items():
    print(f"  {param}:")
    for r in regimes:
        print(f"    [{r.start:>3}-{r.end:>3}]  level={r.level:>6.3f}  se={r.level_se:.3f}")

In [ ]:
fig, ax = result_sis.plot_sis(title="SIS: Intercept Regime Levels")
plt.show()

### OLS Convenience Method

The same analysis can be run via the `.isat()` convenience method on an OLS model:

In [ ]:
model = rg.OLS(y_sis, np.ones((n, 1)), has_constant=False)
result_ols = model.isat(sis=True, alpha=0.01)
print(f"Break dates (via OLS.isat): {result_ols.break_dates}")

## 3. Impulse Indicator Saturation (IIS)

IIS creates a 0/1 impulse indicator for each observation. Retained impulse indicators
identify **one-time outliers** — observations that deviate significantly from the model.

In [ ]:
result_iis = rg.isat(y_iis, iis=True, alpha=0.01)
print(result_iis.summary())

## 4. Combined SIS + IIS

SIS and IIS can be combined to simultaneously detect level shifts and outliers.
This is important when the data contains both types of structural change — ignoring
outliers can distort level shift detection, and vice versa.

In [ ]:
# Add outliers to the mean-shift data
y_combined = y_sis.copy()
y_combined[25] += 8.0
y_combined[175] -= 7.0

result_combined = rg.isat(y_combined, sis=True, iis=True, alpha=0.01)
print(f"Saturation type: {result_combined.saturation_type}")
print(f"Break dates:     {result_combined.break_dates}")
print(f"Retained:        {result_combined.retained_indicators}")

## 5. Multiplicative Indicator Saturation (MIS)

MIS detects **changes in slope coefficients**. For each candidate break date, it creates
step indicators interacted with each regressor: step(t $\geq$ $\tau$) $\times$ x$_j$.
The retained multiplicative indicators identify dates where one or more regression
coefficients change.

In [ ]:
result_mis = rg.isat(y_mis, exog=x_mis, sis=True, mis=True, alpha=0.01)
print(result_mis.summary())

In [ ]:
fig, axes = result_mis.plot_regime_levels(
    title="MIS: Regime Levels for Intercept and Slope"
)
plt.show()

## 6. Trend Indicator Saturation (TIS)

TIS detects **broken linear trends**. For each candidate break date $\tau$, it creates
a piecewise-linear indicator max(0, t $-$ $\tau$) that allows the trend slope to change at $\tau$.

In [ ]:
# Include a linear trend as an exogenous regressor
trend = np.arange(n, dtype=float).reshape(-1, 1)
result_tis = rg.isat(y_tis, exog=trend, tis=True, alpha=0.01)
print(result_tis.summary())

## 7. AR(1) with Dual Structural Breaks

This is the key example. We simulate an AR(1) process with **two types of structural
change at different dates**:

- **t = 100**: intercept shifts from 0 to 2 (level shift)
- **t = 200**: AR coefficient shifts from 0.3 to 0.8 (persistence shift)

SIS + MIS applied jointly should detect both breaks and attribute each to the right
parameter. This demonstrates that GETS can handle **multiple break types at different
times** — something Bai-Perron (which tests all parameters jointly) and Markov switching
(which switches all parameters simultaneously) cannot easily do.

In [ ]:
# Simulate AR(1) with dual breaks
n_ar = 300
y_ar = np.zeros(n_ar)
for t in range(1, n_ar):
    if t < 100:
        c, phi = 0.0, 0.3     # Regime 1: stationary, zero-mean
    elif t < 200:
        c, phi = 2.0, 0.3     # Regime 2: level shift, same dynamics
    else:
        c, phi = 2.0, 0.8     # Regime 3: persistence shift, same intercept
    y_ar[t] = c + phi * y_ar[t - 1] + rng.standard_normal()

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(y_ar, linewidth=0.8)
ax.axvline(100, color="red", linestyle="--", alpha=0.5, label="Intercept shift (t=100)")
ax.axvline(200, color="blue", linestyle="--", alpha=0.5, label="AR coeff shift (t=200)")
ax.set_xlabel("Observation")
ax.set_ylabel("y")
ax.set_title("AR(1) with Dual Structural Breaks")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Run SIS + MIS on the AR(1) model
ar_model = rg.AR(y_ar, lags=1)
result_ar = ar_model.isat(sis=True, mis=True, alpha=0.01)
print(result_ar.summary())

In [ ]:
# Visualize regime levels for both const and AR coefficient
fig, axes = result_ar.plot_regime_levels(
    title="AR(1): Regime Levels for Intercept and AR Coefficient"
)
plt.show()

## 8. ADL Integration

Indicator saturation also works with ADL models via the `.isat()` convenience method.
The lag structure is handled automatically.

In [ ]:
# Simulate ADL(1,0) data with an intercept shift at t=150
x_adl = rng.standard_normal(n)
y_adl = np.zeros(n)
for t in range(1, n):
    c = 1.0 if t < 150 else 3.0
    y_adl[t] = c + 0.4 * y_adl[t - 1] + 0.5 * x_adl[t] + rng.normal(0, 0.5)

adl_model = rg.ADL(y_adl, x_adl, lags=1, exog_lags=0)
result_adl = adl_model.isat(sis=True, alpha=0.01)
print(f"ADL break dates: {result_adl.break_dates}")
print(f"Retained: {result_adl.retained_indicators}")

## 9. Tuning: Significance Level

The `alpha` parameter controls the GETS selection threshold. Lower alpha means fewer
false positives but may miss real breaks; higher alpha is more sensitive but may retain
spurious indicators.

In [ ]:
for a in [0.01, 0.05, 0.10]:
    result = rg.isat(y_sis, sis=True, alpha=a)
    print(f"alpha={a:.2f}:  breaks={result.break_dates}, "
          f"retained={result.n_indicators_retained}")

## 10. Comparison: GETS vs Bai-Perron vs Markov Switching

Three complementary approaches to structural break detection:

| Method | Approach | Break types | Output |
|--------|----------|-------------|--------|
| **GETS/isat** | Indicator selection | Separate per parameter | Break dates + regime levels |
| **Bai-Perron** | Sequential testing | Joint (all params switch) | Break indices + segments |
| **Markov switching** | Latent state model | Probabilistic | Regime probabilities |

In [ ]:
# GETS
result_gets = rg.isat(y_sis, sis=True, alpha=0.01)

# Bai-Perron
bp_test = rg.BaiPerronTest(y_sis)
bp_results = bp_test.fit(max_breaks=5)

# Markov switching
ms_model = rg.MarkovRegression(y_sis, k_regimes=3)
ms_results = ms_model.fit(search_reps=10)

print("GETS (SIS):")
print(f"  Breaks: {result_gets.break_dates}")
print(f"  Regimes: {result_gets.n_regimes}")
print()
print("Bai-Perron:")
print(f"  Breaks: {list(bp_results.break_indices)}")
print(f"  Segments: {bp_results.n_breaks + 1}")
print()
print("Markov Switching (K=3):")
periods = ms_results.regime_periods()
for start, end, regime in periods:
    print(f"  [{start:>3}-{end:>3}] Regime {regime}")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

# GETS
axes[0].plot(y_sis, linewidth=0.8, color="grey", alpha=0.6)
result_gets.plot_sis(ax=axes[0], title="GETS (SIS)")

# Bai-Perron
axes[1].plot(y_sis, linewidth=0.8, color="grey", alpha=0.6)
for bp in bp_results.break_indices:
    axes[1].axvline(bp, color="red", linestyle="--", alpha=0.7)
axes[1].set_title("Bai-Perron")
axes[1].set_ylabel("y")

# Markov switching
ms_results.plot_regime_shading(y=y_sis, ax=axes[2])
axes[2].set_title("Markov Switching (K=3)")

plt.tight_layout()
plt.show()

## 11. End-to-End Workflow

A typical workflow for structural break analysis:

1. **Screen** — CUSUM test for evidence of instability
2. **Detect** — Indicator saturation to locate breaks
3. **Confirm** — Chow test at detected break dates
4. **Estimate** — Final model with confirmed breaks
5. **Diagnose** — Check the final model

In [ ]:
# Step 1: CUSUM screening
model_wf = rg.OLS(y_sis, np.ones((n, 1)), has_constant=False)
cusum_results = model_wf.cusum_test()
print(f"Step 1 - CUSUM rejects stability: {cusum_results.reject}")

# Step 2: Indicator saturation
result_wf = rg.isat(y_sis, sis=True, alpha=0.01)
breaks = result_wf.break_dates
print(f"Step 2 - Detected breaks: {breaks}")

# Step 3: Chow test confirmation
chow_results = model_wf.chow_test(break_points=breaks)
for bp in breaks:
    print(f"Step 3 - Chow test at t={bp}: "
          f"F={chow_results.f_stats[bp]:.2f}, p={chow_results.p_values[bp]:.4f}")

# Step 4: Estimate final model
final_model = rg.OLS(y_sis, np.ones((n, 1)), breaks=breaks, has_constant=False)
final_results = final_model.fit()

# Step 5: Diagnostics
print(f"\nStep 5 - Final model R-squared: {final_results.rsquared:.3f}")
fig, diag_axes = final_results.plot_diagnostics()
plt.show()

## 12. Low-Level GETS Search

For advanced users, `gets_search()` provides direct access to the general-to-specific
algorithm without the indicator saturation wrapper. You supply the full regressor matrix
(including any indicators) and receive the GETS reduction path.

In [ ]:
# Build a custom model with manual step indicators
X_manual = np.column_stack([
    np.ones(n),
    (np.arange(n) >= 100).astype(float),
    (np.arange(n) >= 150).astype(float),
    (np.arange(n) >= 200).astype(float),
    (np.arange(n) >= 250).astype(float),
])
names = ["const", "step_100", "step_150", "step_200", "step_250"]

# Protect the constant from elimination
protected = np.array([True, False, False, False, False])

gets_result = rg.gets_search(
    y_sis, X_manual, names=names, alpha=0.01, protected=protected
)
print(f"Retained: {gets_result.retained_names}")
print(f"Terminals found: {len(gets_result.terminal_models)}")
print(f"Models evaluated: {gets_result.n_models_evaluated}")

## Summary

The `regimes` GETS indicator saturation module provides a complete toolkit for
structural break detection:

| Component | Function / Class |
|-----------|------------------|
| Step indicator saturation (level shifts) | `isat(y, sis=True)` |
| Impulse indicator saturation (outliers) | `isat(y, iis=True)` |
| Multiplicative indicator saturation (coefficient shifts) | `isat(y, exog=X, mis=True)` |
| Trend indicator saturation (broken trends) | `isat(y, exog=trend, tis=True)` |
| Combined saturation | `isat(y, sis=True, iis=True, mis=True)` |
| OLS convenience method | `OLS(y, X).isat(sis=True)` |
| AR convenience method | `AR(y, lags=p).isat(sis=True, mis=True)` |
| ADL convenience method | `ADL(y, x, lags=p).isat(sis=True)` |
| Low-level GETS search | `gets_search(y, X, names, alpha)` |
| Shifts representation | `result.shifts` |
| Regime levels representation | `result.regime_levels` |
| Intercept regime levels plot | `result.plot_sis()` |
| Coefficient regime levels plot | `result.plot_mis()` |
| Combined regime levels plot | `result.plot_regime_levels()` |